In [ ]:
!pip install soundscapy
!pip install soundscapy[audio]
!pip install scipy==1.14.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.8/856.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 7.0 MB/s eta 0:00:00
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160672 sha256=77f9b63d2e8e30303a1b41ccce165f0ee7cf4d19fd10ba5117675b3e7217ce87
  Stored in directory: /root/.cache/pip/wheels/d6/1d/c8/8c29be1d73ca42d15977c75193d9f39a98499413c2838ac54c
Successfully built odfpy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.7/158.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from soundscapy import AudioAnalysis
from soundscapy import AnalysisSettings
from soundscapy import Binaural
from soundscapy import prep_multiindex_df, add_results, process_all_metrics
import json
from pathlib import Path
import sys
from pathlib import Path
sys.path.append('../..')
import warnings
warnings.filterwarnings("ignore")
analysis = AudioAnalysis()

In [ ]:
# Step1: Computing Psychoacoustics from original file
import time
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from soundscapy import Binaural

# Folder path with original wav file
wav_folder = Path("/content/Data")

df = pd.DataFrame()

start = time.perf_counter()

# Loop through each filtered WAV file
for wav in (pbar := tqdm(list(wav_folder.glob("*.wav")))):
    recording = wav.stem
    pbar.set_description(f"Processing {recording}")

        b = Binaural.from_wav(wav)
        # SPL (LZeq)
        spl_result = b.pyacoustics_metric("LZeq", statistics=(5, 50, 'avg', 'max'), as_df=True)

        # Loudness & Sharpness
        metrics = ["loudness_zwtv", "sharpness_din_from_loudness"]
        stats = (5, 50, 'avg', 'max')
        mosqito_results = {}
        for metric in metrics:
            mosqito_results[metric] = b.mosqito_metric(
                metric,
                statistics=stats,
                as_df=True,
                parallel=True,
                func_args={'field_type': 'free'}
            )

        # Combine all metrics (SPL, loudness, and sharpness)
        combined = pd.concat([spl_result] + list(mosqito_results.values()), axis=1)
        combined["Recording"] = recording
        combined = combined.set_index("Recording")

        # Append to master DataFrame
        df = pd.concat([df, combined])

end = time.perf_counter()
print(f" Processing completed in {end - start:.2f} seconds")

print(df.head())

# Save results
df.to_csv("acoustic_metrics_original.csv")

Processing REC566: 100%|██████████| 3/3 [31:37<00:00, 632.36s/it]


✅ Processing completed in 1897.09 seconds
                LZeq     LZeq_5    LZeq_50   LZeq_max        N_5       N_50  \
Recording                                                                     
REC565     72.563131  74.707490  72.124374  76.681324  19.888658  16.950924   
REC565     72.130296  74.730881  71.699823  76.280594  17.808158  15.508664   
REC563     71.372647  74.537626  70.181556  79.901821  20.091380  16.666380   
REC563     71.020903  74.200094  69.874429  79.624749  19.152523  15.976504   
REC566     74.259176  78.813382  72.698698  81.108869  33.472917  17.134173   

               N_avg      N_max        N_5       N_50      N_avg      N_max  \
Recording                                                                     
REC565     17.178956  22.794280  19.888658  16.950924  17.178956  22.794280   
REC565     15.664634  21.046655  17.808158  15.508664  15.664634  21.046655   
REC563     16.872344  34.355973  20.091380  16.666380  16.872344  34.355973   
REC563  

In [ ]:
# Step2: Apply band filter to approximate AMM effects on 200 Hz
import numpy as np
import scipy.signal as signal
import librosa
import librosa.display
import matplotlib.pyplot as plt
import soundfile as sf
from pathlib import Path
from tqdm import tqdm
from scipy.signal import iirnotch, lfilter
import pandas as pd

def compute_q_factor(target_reduction_db, min_q=0.6, max_q=3.0):
    return np.clip(9 / (target_reduction_db - 9), min_q, max_q)

# Notch Filter customization
def notch_filter_with_db_reduction(data, center_freq, fs, attenuation_db):
    nyquist = 0.5 * fs
    quality_factor = compute_q_factor(attenuation_db)
    b, a = iirnotch(w0=center_freq / nyquist, Q=quality_factor)

    filtered_data = np.copy(data)
    for i in range(filtered_data.shape[0]):
        filtered_data[i] = lfilter(b, a, filtered_data[i])
    return filtered_data

# Input Paths
input_folder = Path("/content/Data")
output_folder = input_folder / "Filtered"
output_folder.mkdir(exist_ok=True)

# Load attenuation values from CSV- attenuation db
attenuation_csv = input_folder / "AMM_9_to_python.csv"
attenuation_df = pd.read_csv(attenuation_csv)
attenuation_map = dict(zip(attenuation_df["id"], attenuation_df["attenuation_db"]))

# Process Each WAV File
for wav_file in (pbar := tqdm(list(input_folder.glob("*.wav")))):
    recording_id = wav_file.stem
    pbar.set_description(f"Filtering {recording_id}")
    # Load WAV
    y, sr = librosa.load(str(wav_file), sr=None, mono=False)
    if y.ndim == 1:
        y = np.vstack([y, y])
    elif y.shape[1] == 2:
        y = y.T

    # Get attenuation dB for this file
    attenuation_db = attenuation_map[recording_id]

    # Apply filter
    y_filtered = notch_filter_with_db_reduction(
        y, center_freq=200, fs=sr, attenuation_db=attenuation_db
    )

    # Save filtered WAV
    y_filtered = y_filtered.T
    filtered_path = output_folder / f"{recording_id}_filtered.wav"
    sf.write(str(filtered_path), y_filtered.astype(np.float32), sr, format="WAV", subtype="FLOAT")

    print(f"Saved: {filtered_path.name} | Applied {attenuation_db} dB notch")



Filtering REC563:  33%|███▎      | 1/3 [00:00<00:00,  3.36it/s]

Saved: REC565_filtered.wav | Applied 21.86246544 dB notch


Filtering REC566:  67%|██████▋   | 2/3 [00:00<00:00,  3.46it/s]

Saved: REC563_filtered.wav | Applied 23.92516811 dB notch


Filtering REC566: 100%|██████████| 3/3 [00:00<00:00,  3.49it/s]

Saved: REC566_filtered.wav | Applied 26.37720339 dB notch


In [ ]:
# Step3: Computing Psychoacoustics from filter wav file
import time
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from soundscapy import Binaural

# folder with filtered & calibrated WAVs
wav_folder = Path("/content/Data/Filtered")

# Initialize empty DataFrame to store results
df = pd.DataFrame()

# Start timer
start = time.perf_counter()

# Loop through each filtered WAV file
for wav in (pbar := tqdm(list(wav_folder.glob("*_filtered.wav")))):
    recording = wav.stem.replace("_filtered", "")
    pbar.set_description(f"Processing {recording}")

    try:
        # Load WAV without additional calibration——already calibrated
        b = Binaural.from_wav(wav)

        # SPL metric
        spl_result = b.pyacoustics_metric("LZeq", statistics=(5, 50, 'avg', 'max'), as_df=True)

        # Loudness & Sharpness
        metrics = ["loudness_zwtv", "sharpness_din_from_loudness"]
        stats = (5, 50, 'avg', 'max')
        mosqito_results = {}
        for metric in metrics:
            mosqito_results[metric] = b.mosqito_metric(
                metric,
                statistics=stats,
                as_df=True,
                parallel=True,
                func_args={'field_type': 'free'}
            )

        # Combine all metrics
        combined = pd.concat([spl_result] + list(mosqito_results.values()), axis=1)
        combined["Recording"] = recording
        combined = combined.set_index("Recording")

        # Append to main DataFrame
        df = pd.concat([df, combined])

end = time.perf_counter()
print(f"\n Done in {end - start:.2f} seconds")

print(df.head())

#  Save to CSV
df.to_csv("acoustic_metrics_filtered.csv")
print(" Results saved to 'acoustic_metrics_filtered.csv'")

Processing REC563: 100%|██████████| 3/3 [32:53<00:00, 657.86s/it]


✅ Done in 1973.62 seconds
                LZeq     LZeq_5    LZeq_50   LZeq_max        N_5       N_50  \
Recording                                                                     
REC566     72.826421  77.815355  71.013507  80.519591  30.044983  14.929338   
REC566     73.361556  79.725577  70.701344  82.803009  32.556981  14.883555   
REC565     71.047953  73.406708  70.738207  74.850332  18.043653  15.236251   
REC565     70.696881  73.209913  70.365561  74.981476  15.886111  13.863265   
REC563     69.141593  71.787980  68.448424  76.272476  17.630266  14.721076   

               N_avg      N_max        N_5       N_50      N_avg      N_max  \
Recording                                                                     
REC566     17.675391  36.108672  30.044983  14.929338  17.675391  36.108672   
REC566     17.799556  40.737225  32.556981  14.883555  17.799556  40.737225   
REC565     15.458070  20.935773  18.043653  15.236251  15.458070  20.935773   
REC565     14.002778  19